In [1]:
%cd ..

/home/kuba/Projects/github_search


In [2]:
import requests
from io import StringIO
import sys
import time
import tqdm

import pypi_cli
from sklearn import feature_extraction, metrics
import numpy as np
import pandas as pd
import bs4

import mlutil.parallel

import elasticsearch
import haystack.document_store.memory
import haystack.document_store.elasticsearch
from haystack import document_store

import haystack.retriever.sparse 
from haystack import retriever


from github_search import repository_descriptions

In [3]:
!wc -l ../data/python/train.jsonl

wc: ../data/python/train.jsonl: No such file or directory


In [4]:
!head  -1 ../data/python/train.jsonl

head: cannot open '../data/python/train.jsonl' for reading: No such file or directory


In [5]:
all_codesearch_df = repository_descriptions.get_all_codesearch_df('data/python')

In [6]:
all_codesearch_df['repo'].value_counts().index[:20]

Index(['saltstack/salt', 'mitsei/dlkit', 'google/grr', 'bcbio/bcbio-nextgen',
       'materialsproject/pymatgen', 'tensorflow/tensor2tensor',
       'iotile/coretools', 'pandas-dev/pandas', 'cloud9ers/gurumate',
       'spyder-ide/spyder', 'pypa/pipenv', 'apple/turicreate', 'gem/oq-engine',
       'pantsbuild/pants', 'log2timeline/plaso',
       'googleapis/google-cloud-python', 'inasafe/inasafe', 'gwastro/pycbc',
       'apache/incubator-mxnet', 'senaite/senaite.core'],
      dtype='object')

In [7]:
repos_descriptions_df = repository_descriptions.load_pypi_repo_descriptions()

In [8]:
repos_descriptions_df.head()

,repo,pypi_description
Unnamed: 0,,
0,smdabdoub/phylotoast,Tools for phylogenetic data analysis including visualization and cluster-com...
1,mkouhei/bootstrap-py,Open-source algorithms for data-driven building analysis and control
2,elbow-jason/Uno-deprecated,Bootstrap Python package
3,disqus/nydus,Extremely fast and easy feature based HTML generator.
4,jay-johnson/network-pipeline,Connection utilities


In [9]:
all_codesearch_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,smdabdoub/phylotoast,phylotoast/util.py,split_phylogeny,"def split_phylogeny(p, level=""s""):\n """"""\n Return either the full or t...",python,"def split_phylogeny(p, level=""s""):\n """"""\n Return either the full or t...","[def, split_phylogeny, (, p, ,, level, =, ""s"", ), :, level, =, level, +, ""__...",Return either the full or truncated version of a QIIME-formatted taxonomy st...,"[Return, either, the, full, or, truncated, version, of, a, QIIME, -, formatt...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
1,smdabdoub/phylotoast,phylotoast/util.py,ensure_dir,"def ensure_dir(d):\n """"""\n Check to make sure the supplied directory p...",python,"def ensure_dir(d):\n """"""\n Check to make sure the supplied directory p...","[def, ensure_dir, (, d, ), :, if, not, os, ., path, ., exists, (, d, ), :, t...","Check to make sure the supplied directory path does not exist, if so, create...","[Check, to, make, sure, the, supplied, directory, path, does, not, exist, if...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
2,smdabdoub/phylotoast,phylotoast/util.py,file_handle,"def file_handle(fnh, mode=""rU""):\n """"""\n Takes either a file path or a...",python,"def file_handle(fnh, mode=""rU""):\n """"""\n Takes either a file path or a...","[def, file_handle, (, fnh, ,, mode, =, ""rU"", ), :, handle, =, None, if, isin...","Takes either a file path or an open file handle, checks validity and returns...","[Takes, either, a, file, path, or, an, open, file, handle, checks, validity,...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
3,smdabdoub/phylotoast,phylotoast/util.py,gather_categories,"def gather_categories(imap, header, categories=None):\n """"""\n Find the...",python,"def gather_categories(imap, header, categories=None):\n """"""\n Find the...","[def, gather_categories, (, imap, ,, header, ,, categories, =, None, ), :, #...",Find the user specified categories in the map and create a dictionary to con...,"[Find, the, user, specified, categories, in, the, map, and, create, a, dicti...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
4,smdabdoub/phylotoast,phylotoast/util.py,parse_unifrac,"def parse_unifrac(unifracFN):\n """"""\n Parses the unifrac results file ...",python,"def parse_unifrac(unifracFN):\n """"""\n Parses the unifrac results file ...","[def, parse_unifrac, (, unifracFN, ), :, with, open, (, unifracFN, ,, ""rU"", ...",Parses the unifrac results file into a dictionary\n\n :type unifracFN: st...,"[Parses, the, unifrac, results, file, into, a, dictionary]",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train


## Baseline - retrieval by bag of words from descriptions

Fit bag of words model to descriptions, then match 'descriptions' obtained from concatenating comments from all functions from repository

The assumptions here are very optimistic, since we're only matching features from known repositories 

In [10]:
all_codesearch_df.columns

Index(['repo', 'path', 'func_name', 'original_string', 'language', 'code',
       'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url',
       'partition'],
      dtype='object')

In [11]:
all_codesearch_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,smdabdoub/phylotoast,phylotoast/util.py,split_phylogeny,"def split_phylogeny(p, level=""s""):\n """"""\n Return either the full or t...",python,"def split_phylogeny(p, level=""s""):\n """"""\n Return either the full or t...","[def, split_phylogeny, (, p, ,, level, =, ""s"", ), :, level, =, level, +, ""__...",Return either the full or truncated version of a QIIME-formatted taxonomy st...,"[Return, either, the, full, or, truncated, version, of, a, QIIME, -, formatt...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
1,smdabdoub/phylotoast,phylotoast/util.py,ensure_dir,"def ensure_dir(d):\n """"""\n Check to make sure the supplied directory p...",python,"def ensure_dir(d):\n """"""\n Check to make sure the supplied directory p...","[def, ensure_dir, (, d, ), :, if, not, os, ., path, ., exists, (, d, ), :, t...","Check to make sure the supplied directory path does not exist, if so, create...","[Check, to, make, sure, the, supplied, directory, path, does, not, exist, if...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
2,smdabdoub/phylotoast,phylotoast/util.py,file_handle,"def file_handle(fnh, mode=""rU""):\n """"""\n Takes either a file path or a...",python,"def file_handle(fnh, mode=""rU""):\n """"""\n Takes either a file path or a...","[def, file_handle, (, fnh, ,, mode, =, ""rU"", ), :, handle, =, None, if, isin...","Takes either a file path or an open file handle, checks validity and returns...","[Takes, either, a, file, path, or, an, open, file, handle, checks, validity,...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
3,smdabdoub/phylotoast,phylotoast/util.py,gather_categories,"def gather_categories(imap, header, categories=None):\n """"""\n Find the...",python,"def gather_categories(imap, header, categories=None):\n """"""\n Find the...","[def, gather_categories, (, imap, ,, header, ,, categories, =, None, ), :, #...",Find the user specified categories in the map and create a dictionary to con...,"[Find, the, user, specified, categories, in, the, map, and, create, a, dicti...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train
4,smdabdoub/phylotoast,phylotoast/util.py,parse_unifrac,"def parse_unifrac(unifracFN):\n """"""\n Parses the unifrac results file ...",python,"def parse_unifrac(unifracFN):\n """"""\n Parses the unifrac results file ...","[def, parse_unifrac, (, unifracFN, ), :, with, open, (, unifracFN, ,, ""rU"", ...",Parses the unifrac results file into a dictionary\n\n :type unifracFN: st...,"[Parses, the, unifrac, results, file, into, a, dictionary]",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0b74ef171e6a84761710548501dfac7...,train


In [12]:
all_codesearch_df.groupby('repo').agg('count')['url']

repo
0101/pipetools                    8
02strich/django-auth-kerberos     1
05bit/peewee-async               30
0compute/xtraceback               2
0k/kids.cache                     2
                                 ..
zyga/padme                        2
zyga/python-glibc                24
zyga/python-phablet               2
zzzsochi/includer                 2
zzzsochi/resolver_deco            1
Name: url, Length: 12361, dtype: int64

In [13]:
len(repos_descriptions_df)

8940

In [14]:
codesearch_df = all_codesearch_df[all_codesearch_df['repo'].isin(repos_descriptions_df['repo'])]

In [15]:
train_codesearch_df = codesearch_df[codesearch_df['partition'] == 'train']
val_codesearch_df = codesearch_df[codesearch_df['partition'] == 'val']

In [16]:
set(all_codesearch_df['partition'].values)

{'test', 'train', 'valid'}

In [28]:
def get_tfidf_retriever(df, col, elasticsearch_client=elasticsearch.Elasticsearch()):
    memory_docstring_store = document_store.elasticsearch.ElasticsearchDocumentStore(index=col)
    df['text'] = df[col]
    if not elasticsearch_client.count(index=col).get('count'):
        memory_docstring_store.write_documents(df.to_dict('records'))
    return retriever.sparse.ElasticsearchRetriever(document_store=memory_docstring_store)

In [29]:
%%time
docstring_tfidf_retriever = get_tfidf_retriever(codesearch_df, 'docstring')

<ipython-input-28-4a9075ba1e32>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df[col]


CPU times: user 24.1 s, sys: 316 ms, total: 24.4 s
Wall time: 7min 8s


In [30]:
def get_repos(ret, description, top_k=100):
    return [rec.meta['repo'] for rec in ret.retrieve(query=description, top_k=top_k)]

In [31]:
aggregated_docstrings = codesearch_df.groupby('repo')[['docstring']].agg(' '.join)
aggregated_docstrings['repo'] = aggregated_docstrings.index
aggregated_docstrings['aggregated_docstring'] = aggregated_docstrings['docstring']

In [32]:
docstring_matched_repos = [
    get_repos(docstring_tfidf_retriever, descr)
    for descr in tqdm.tqdm(repos_descriptions_df['pypi_description'])
]

100%|██████████| 8940/8940 [03:00<00:00, 49.56it/s]


In [33]:
texts = [
    doc.text
    for doc in tqdm.tqdm(docstring_tfidf_retriever.document_store.get_all_documents_generator())
]

196876it [00:09, 20154.29it/s]


In [40]:
len(docstring_matched_repos)

8940

In [41]:
recall_at100= [
    repo in potential_repos
    for (repo, potential_repos) in zip(repos_descriptions_df['repo'], docstring_matched_repos)
]

In [43]:
sum(recall_at100) / len(docstring_matched_repos)

0.007941834451901567

## Pomysły

- jako baseline wyszukiwanie zanurzeń komentarzy
- wyszukiwanie po cechach korzystających z CodeBERTA
- wytrenowanie modelu do rankingu (?) 

### Ogólne pomysły na podejścia

Problem wydaje się dobrze opisywać podejście 'pairwise': mamy cechy jednego typu i drugiego typu, uczymy się dopasowywać jedne do drugich

- ograniczyć się do komentarzy/kodu z wyższego poziomu hierarchii (wywołań/zależności)
- agregowanie cech kodu z repozytoriów
- agregowanie cech pochodzących od importów (Import2Vec) - **pokaż mi co importujesz, a powiem na jaki temat jest repo**
- ogólniej cechy dla specjalnych tokenów: nazwy funkcji, nazwy argumentów
- sprytniejsza agregacja - np hierarchiczna (np zrobić hiperboliczną wersję zanurzeń)